In [2]:
import subprocess
import myvariant
import os
import sys
import bisect
import pyranges as pr
from cyvcf2 import VCF, Writer
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [3]:
#all_Omics_files = pd.read_csv(f'/sbgenomics/workspace/Tables/all_Omics_file_index_202511.csv', index_col=0)
all_Omics_files = pd.read_csv(f'/sbgenomics/workspace/Tables/all_Omics_file_index_202601.csv', index_col=0)
print(all_Omics_files.shape)
all_Omics_files.head()

(185985, 6)


,dbGAP_id,dataset,file_name,extension,size_MB,path
0,phs000974-v6-c1,nih-nhlbi-topmed-freeze10-phs000974-v6-c1,Release_Notes.phs000974.TOPMed_WGS_Framingham....,pdf,0.449,/sbgenomics/project-files/Omics/nih-nhlbi-topm...
1,phs000974-v6-c1,nih-nhlbi-topmed-freeze10-phs000974-v6-c1,Study_Report.phs000974.TOPMed_WGS_Framingham.v...,pdf,0.137,/sbgenomics/project-files/Omics/nih-nhlbi-topm...
2,phs000974-v6-c1,nih-nhlbi-topmed-freeze10-phs000974-v6-c1,manifest_phs000974.TOPMed_WGS_Framingham.v6.p5...,pdf,0.047,/sbgenomics/project-files/Omics/nih-nhlbi-topm...
3,phs000974-v6-c1,nih-nhlbi-topmed-freeze10-phs000974-v6-c1,phs000974.v6.pht004909.v5.TOPMed_FHS_Subject.d...,xml,0.001,/sbgenomics/project-files/Omics/nih-nhlbi-topm...
4,phs000974-v6-c1,nih-nhlbi-topmed-freeze10-phs000974-v6-c1,phs000974.v6.pht004909.v5.p5.TOPMed_FHS_Subjec...,txt.gz,0.038,/sbgenomics/project-files/Omics/nih-nhlbi-topm...


In [9]:
#study = 'parent-CHS'
#study = 'parent-MESA'
#study = 'topmed-CHS'
#study = 'topmed-JHS'
#study = 'topmed-MESA'
study = 'topmed-FHS'
id = 'c2'
select_files = all_Omics_files[all_Omics_files['dataset'] == f'nih-nhlbi-topmed-freeze10-phs000974-v6-c2']
select_files = select_files[select_files['extension'] == 'vcf.gz']
print(select_files.shape)
select_files.head()

(24, 6)


,dbGAP_id,dataset,file_name,extension,size_MB,path
3793,phs000974-v6-c2,nih-nhlbi-topmed-freeze10-phs000974-v6-c2,FRAMINGHAM_phs000974_TOPMed_WGS_freeze.10b.chr...,vcf.gz,1186.074,/sbgenomics/project-files/Omics/nih-nhlbi-topm...
3795,phs000974-v6-c2,nih-nhlbi-topmed-freeze10-phs000974-v6-c2,FRAMINGHAM_phs000974_TOPMed_WGS_freeze.10b.chr...,vcf.gz,710.615,/sbgenomics/project-files/Omics/nih-nhlbi-topm...
3797,phs000974-v6-c2,nih-nhlbi-topmed-freeze10-phs000974-v6-c2,FRAMINGHAM_phs000974_TOPMed_WGS_freeze.10b.chr...,vcf.gz,726.487,/sbgenomics/project-files/Omics/nih-nhlbi-topm...
3799,phs000974-v6-c2,nih-nhlbi-topmed-freeze10-phs000974-v6-c2,FRAMINGHAM_phs000974_TOPMed_WGS_freeze.10b.chr...,vcf.gz,697.103,/sbgenomics/project-files/Omics/nih-nhlbi-topm...
3801,phs000974-v6-c2,nih-nhlbi-topmed-freeze10-phs000974-v6-c2,FRAMINGHAM_phs000974_TOPMed_WGS_freeze.10b.chr...,vcf.gz,510.269,/sbgenomics/project-files/Omics/nih-nhlbi-topm...


In [6]:
genes_df = pd.read_csv("/sbgenomics/workspace/Tables/ref/hg38.refGene.genes.bed", sep="\t", names=["chrom", "start", "end", "gene"]).sort_values(["chrom", "start"])
genes_df

,chrom,start,end,gene
0,chr1,11873,14409,DDX11L1
1,chr1,14361,29370,WASH7P
2,chr1,17368,17436,MIR6859-1
3,chr1,17368,17436,MIR6859-2
4,chr1,17368,17436,MIR6859-3
...,...,...,...,...
24904,chrY,57184215,57197337,IL9R
73013,chrY,57184215,57197337,IL9R
73695,chrY,57201083,57203350,WASIR1
73696,chrY,57212177,57214703,DDX11L16


In [11]:
## ======================================================
## Integrating bcftools
## ======================================================

log_path = f"/sbgenomics/workspace/Tables/VCF/{study}/{id}/output_1.txt"
os.makedirs(os.path.dirname(log_path), exist_ok=True)

if os.path.exists(log_path):
    os.remove(log_path)

log_fh = open(log_path, "w")  ## append mode

def fprint(*args, **kwargs):
    print(*args, **kwargs)
    print(*args, **kwargs, file=log_fh)
    log_fh.flush()

count = 0
for file_id in tqdm(select_files.index[:], desc='Files'):
    count += 1
    fprint(f"*********************{count}/{select_files.shape[0]} {select_files.loc[file_id, 'file_name'][:-7]}*********************")
    
    path = select_files.loc[file_id, 'path']
    if "," in path:
        path = path.split(",")[0].strip().strip("'").strip('"')

    out_dir = f"/sbgenomics/workspace/Tables/VCF/{study}/{id}"
    
    step1_vcf = f"{out_dir}/step1_1.vcf.gz"
    step2_vcf = f"{out_dir}/step2_1.vcf.gz"
    final_vcf = f"{out_dir}/step3_1.vcf.gz"

    ## ======================================================
    ## STEP 1 — Basic quality filtering (bcftools)
    ## QUAL >= 30, FILTER == PASS
    ## ======================================================
    subprocess.run(
        [
            "bcftools", "view",
            "-i", 'QUAL>=30 && FILTER="PASS"',
            "-Oz",
            "-o", step1_vcf,
            path
        ],
        check=True
    )
    subprocess.run(["bcftools", "index", "-f", step1_vcf], check=True)
    
    
    ## ======================================================
    ## STEP 2 — Normalize & split multiallelics (bcftools)
    ## Equivalent to your Step 2a + 2b
    ## ======================================================
    subprocess.run(
        [
            "bcftools", "norm",
            "-m", "-any",           # split multi-allelics
            "-Oz",
            "-o", step2_vcf,
            step1_vcf
        ],
        check=True
    )
    subprocess.run(["bcftools", "index", "-f", step2_vcf], check=True)
    
    
    ## ======================================================
    ## STEP 3 — Annotation-based filtering (Python)
    ## AF <= 0.01, missense / stop_gained, SNV only
    ## ======================================================
    vcf = VCF(step2_vcf)
    writer = Writer(final_vcf, vcf)

    fprint("Number of samples:", len(vcf.samples))

    kept = 0
    for v in tqdm(vcf, leave=False, desc="Step 3 (Python)"):
        ## SNV only (safety check)
        if len(v.REF) != 1 or any(len(a) != 1 for a in v.ALT):
            continue

        af = v.INFO.get("AF")
        if af is not None:
            if isinstance(af, (list, tuple)):
                af = min(af)
            if af > 0.01:
                continue

        ann = v.INFO.get("ANN", "")
        if "missense" not in ann and "stop_gained" not in ann:
            continue

        writer.write_record(v)
        kept += 1

    writer.close()
    vcf.close()
    fprint(f"Final variants kept: {kept}")


    
    ## Output
    vcf = VCF(f"/sbgenomics/workspace/Tables/VCF/{study}/{id}/step3_1.vcf.gz")
    
    gene_index = {}
    for chrom, sub in genes_df.groupby("chrom"):
        gene_index[chrom] = {
            "starts": sub.start.tolist(),
            "ends": sub.end.tolist(),
            "genes": sub.gene.tolist()
        }
    
    def find_gene(chrom, pos0):
        if chrom not in gene_index:
            return None
        starts = gene_index[chrom]["starts"]
        ends = gene_index[chrom]["ends"]
        genes = gene_index[chrom]["genes"]
        i = bisect.bisect_right(starts, pos0) - 1
        if i >= 0 and pos0 < ends[i]:
            return genes[i]
        return None
    
    def encode_gt(gt):
        """
        gt is like [a, b, phased]
        """
        a, b = gt[0], gt[1]
        if a == -1 or b == -1:
            return None
        return a + b
    
    samples = vcf.samples
    
    records = []
    for v in tqdm(vcf, leave=False, desc='Output'):
        row = {
            "chrom": v.CHROM,
            "pos": v.POS,
            "ref": v.REF,
            "alt": v.ALT[0],
            "qual": v.QUAL,
            "gene": find_gene(v.CHROM, v.POS - 1),
            #"dp": v.INFO.get("DP"),
            #"af": v.INFO.get("AF"),
            "dbsnp_id": v.ID if v.ID != "." else None
        }
        ## Add one column per sample
        for sample, gt in zip(samples, v.genotypes):
            row[sample] = encode_gt(gt)
        records.append(row)
    vcf.close()
    
    df = pd.DataFrame(records)
    fprint(f"Final output (all SNPs): {df.shape}")
    df = df[df.iloc[:,7:].sum(axis=1) >0]
    fprint(f"Final output (remove non-idendified SNPs): {df.shape}")

    ## dbSNP annotation
    try:
        ID_list = df['dbsnp_id'].values
        selected_columns = [
            "_id",
            "dbsnp.rsid",
            "dbsnp.chrom",
            "dbsnp.hg19.start",
            "dbsnp.hg19.end",
            "dbsnp.ref",
            "dbsnp.alt",
            "dbsnp.vartype",
            "dbsnp.gene.symbol",
            "snpeff.ann.effect",
            "snpeff.ann.feature_id",
            "snpeff.ann.feature_type",
            "snpeff.ann.genename",
            "snpeff.ann.gene_id",
            "snpeff.ann.putative_impact",
            "snpeff.ann.transcript_biotype",
            "cadd.phred",
            "cadd.rawscore",
            "cadd.consequence",
            #"cadd.fitcons",
            #"cadd.gerp.s"
        ]
        mv = myvariant.MyVariantInfo()
        df_ids = mv.getvariants(ID_list, fields="dbsnp,snpeff,cadd,clinvar", as_dataframe=True)
        df_id_filter = df_ids[selected_columns]
        keep_terms = ['missense_variant', 'stop_gained']
        df_id_filter_filter = df_id_filter[df_id_filter['snpeff.ann.effect'].isin(keep_terms)]
        df = df[df['dbsnp_id'].isin(set(df_id_filter_filter.index))]
        fprint(f"Final output (After dbSNP annotation and filtering): {df.shape}")
        rows = []
        for i in tqdm(df.index, leave=False, desc="dbSNP annotation"):
            row_id = df.loc[i, 'dbsnp_id']
            #data = df_id_filter_filter[df_id_filter_filter['query'] == row_id]
            data = df_id_filter_filter.loc[[row_id]]
            rows.append([data['snpeff.ann.effect'].values[0], data['dbsnp.gene.symbol'].values[0]])
        rows = np.asarray(rows)
        if "var_type" in df.columns:
            df["var_type"] = rows[:, 0]
        else:
            df.insert(7, "var_type", rows[:, 0])
        if "gene_dbsnp" in df.columns:
            df["gene_dbsnp"] = rows[:, 1]
        else:
            df.insert(8, "gene_dbsnp", rows[:, 1])
        
        df.to_csv(f"/sbgenomics/workspace/Tables/VCF/{study}/{id}/{select_files.loc[file_id, 'file_name'][:-7]}.csv", index=True)
    except:
        fprint("No SNP identified!")
    
    os.remove(f"/sbgenomics/workspace/Tables/VCF/{study}/{id}/step1_1.vcf.gz")
    os.remove(f"/sbgenomics/workspace/Tables/VCF/{study}/{id}/step2_1.vcf.gz")
    os.remove(f"/sbgenomics/workspace/Tables/VCF/{study}/{id}/step3_1.vcf.gz")
    os.remove(f"/sbgenomics/workspace/Tables/VCF/{study}/{id}/step1_1.vcf.gz.csi")
    os.remove(f"/sbgenomics/workspace/Tables/VCF/{study}/{id}/step2_1.vcf.gz.csi")

log_fh.close()

Files:   0%|          | 0/5 [00:00<?, ?it/s]

*********************1/24 FRAMINGHAM_phs000974_TOPMed_WGS_freeze.10b.chr1.hg38.c2*********************


Lines   total/split/joined/realigned/skipped:	3123650/0/0/0/0


Number of samples: 883


Step 3 (Python): 0it [00:00, ?it/s]

...

Final variants kept: 3780


Output: 0it [00:00, ?it/s]

Input sequence provided is already in string format. No operation performed


Final output (all SNPs): (3780, 890)
Final output (remove non-idendified SNPs): (3780, 890)
Final output (After dbSNP annotation and filtering): (1396, 890)


dbSNP annotation:   0%|          | 0/1396 [00:00<?, ?it/s]